In [7]:
# | default_exp train.nbeats

In [8]:
# | export
from datasetsforecast.m5 import M5

# , group="Monthly"
df = M5().load("../data")[0]
df.sort_values(["unique_id", "ds"], inplace=True)
df

,unique_id,ds,y
0,FOODS_1_001_CA_1,2011-01-29,3.0
1,FOODS_1_001_CA_1,2011-01-30,0.0
2,FOODS_1_001_CA_1,2011-01-31,0.0
3,FOODS_1_001_CA_1,2011-02-01,1.0
4,FOODS_1_001_CA_1,2011-02-02,4.0
...,...,...,...
47735392,HOUSEHOLD_2_516_WI_3,2016-06-15,0.0
47735393,HOUSEHOLD_2_516_WI_3,2016-06-16,1.0
47735394,HOUSEHOLD_2_516_WI_3,2016-06-17,0.0
47735395,HOUSEHOLD_2_516_WI_3,2016-06-18,0.0


In [9]:
df.unique_id.nunique()

30490

In [10]:
df.groupby("unique_id").apply(len).describe()

/tmp/ipykernel_67123/840638693.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("unique_id").apply(len).describe()
/tmp/ipykernel_67123/840638693.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby("unique_id").apply(len).describe()


count    30490.000000
mean      1562.805510
std        477.176658
min        124.000000
25%       1203.000000
50%       1810.000000
75%       1968.000000
max       1969.000000
dtype: float64

In [11]:
# | export
from ts.preprocess.dataloader import UnivariateTSDataModule

In [12]:
# | export

horizon = 12
input_size = horizon * 5
batch_size = 512
num_workers = 24
step_size = 3

ds = UnivariateTSDataModule(
    df=df,
    input_size=input_size,
    horizon=horizon,
    batch_size=batch_size,
    num_workers=num_workers,
    train_split=0.7,
    val_split=0.15,
    normalize=True,
    scaler_type="minmax",
    split_type="vertical",
    step_size=step_size,
)

In [13]:
# | export
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

from ts.model.nbeats import NBeatsG

In [14]:
# | export
# Example trainer setup (without full NBeatsG for brevity)

import torch
from pytorch_lightning.profilers import PyTorchProfiler

torch.autograd.set_detect_anomaly(True)

profiler = PyTorchProfiler(
    profile_memory=True,  # Track GPU memory
    record_shapes=True,
    with_stack=True,  # Track CPU memory (if supported)
)

trainer = pl.Trainer(
    max_epochs=200,  # Short run for testing
    accelerator="auto",
    logger=TensorBoardLogger("M4_logs", name="nbeatsg_m3_monthly"),
    callbacks=[EarlyStopping("val_loss", patience=20, verbose=False)],
    profiler=profiler,
)

model = NBeatsG(input_size=input_size, horizon=horizon)
trainer.fit(model, ds)

INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/home/pranav-pc/projects/ts/ts/preprocess/dataloader.py:210: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = self.df.groupby("unique_id")
INFO:ts.preprocess.dataloader:Train windows: 10452783, Val windows: 2243024, Test windows: 2232553
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/pranav-pc/projects/ts/.venv/lib/python3.12/site-packa

Sanity Checking: |                                                                                            …

[W329 22:03:23.678259948 CPUAllocator.cpp:245] Memory block of unknown size was allocated before the profiling started, profiler results will not include the deallocation event


Training: |                                                                                                   …

INFO:pytorch_lightning.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [ ]:
# | export
trainer.test(model, ds);

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import torch
from plotly.subplots import make_subplots


def forecast_and_plot_grid(trainer, model, data_module, num_series=6):
    # Ensure model is in evaluation mode
    model.eval()

    # Get all unique_ids from the DataFrame
    unique_ids = data_module.df["unique_id"].unique()
    # Randomly select 6 unique_ids (or fewer if less available)
    selected_ids = np.random.choice(
        unique_ids, size=min(num_series, len(unique_ids)), replace=False
    )

    # Prepare test data for selected series
    test_data = data_module.df[data_module.df["unique_id"].isin(selected_ids)]
    grouped = test_data.groupby("unique_id")

    # Create figure with 3x2 grid
    fig = make_subplots(
        rows=3,
        cols=2,
        subplot_titles=[f"Series: {uid}" for uid in selected_ids],
        vertical_spacing=0.15,
        horizontal_spacing=0.1,
    )

    # Perform forecasting and plotting
    for i, unique_id in enumerate(selected_ids):
        series_df = grouped.get_group(unique_id)
        series = series_df["y"].values
        series_len = len(series)

        if series_len < data_module.input_size + data_module.horizon:
            print(f"Skipping {unique_id}: too short for forecasting")
            continue

        # Normalize series if data_module uses normalization
        if data_module.normalize:
            scaler = data_module.scalers.get(unique_id)
            if scaler:
                series_normalized = scaler.transform(series.reshape(-1, 1)).flatten()
            else:
                series_normalized = series
        else:
            series_normalized = series

        # Generate last input window for forecasting
        last_input = series_normalized[
            -data_module.input_size - data_module.horizon : -data_module.horizon
        ]
        x = torch.tensor(last_input, dtype=torch.float32).unsqueeze(0).to(model.device)

        # Forecast
        with torch.no_grad():
            y_hat = model(x).cpu().numpy().flatten()

        # Inverse transform predictions and actual values
        if data_module.normalize and scaler:
            y_hat_denorm = scaler.inverse_transform(y_hat.reshape(-1, 1)).flatten()
            series_denorm = series  # Already in original scale
        else:
            y_hat_denorm = y_hat
            series_denorm = series

        # Time indices for the entire series and forecast
        full_time_indices = series_df["ds"].values  # Full series dates
        forecast_time_indices = full_time_indices[-data_module.horizon :]  # Last horizon dates

        # Determine row and column (0-based indexing, converting to 1-based for Plotly)
        row = (i // 2) + 1
        col = (i % 2) + 1

        # Plot the entire actual series
        fig.add_trace(
            go.Scatter(
                x=full_time_indices,
                y=series_denorm,
                mode="lines+markers",
                line=dict(color="blue"),
                showlegend=False,
            ),
            row=row,
            col=col,
        )

        # Plot predicted values for the last horizon
        fig.add_trace(
            go.Scatter(
                x=forecast_time_indices,
                y=y_hat_denorm,
                mode="lines+markers",
                line=dict(color="red", dash="dash"),
                showlegend=False,
            ),
            row=row,
            col=col,
        )

    # Update layout
    fig.update_layout(
        height=900,
        width=800,  # Fixed size for 3x2 grid
        title_text="N-BEATS-G Forecasting: Full Series with Last Horizon Predicted (3x2 Grid)",
        showlegend=False,  # Remove legend entirely
    )
    fig.update_yaxes(title_text="Value")
    fig.update_xaxes(title_text="Date")

    # Show plot
    fig.show()


# Example usage (assuming trainer, model, and data_module are defined)
# Replace 'ds' with 'data_module' if that’s your variable name
forecast_and_plot_grid(trainer, model, ds, num_series=6)

In [ ]:
# | export
trainer.save_checkpoint("M5-MONTHLY-profile.ckpt")